# pyAutoTradingBot

## Open Trade Strategy - Import Data

In [1]:
import os
import pandas as pd
from PIL import Image, ImageDraw

In [2]:

csv_diario = pd.read_csv(os.path.join('input','WDOFUT_F_0_Diario.train.csv'), sep=';')
csv_15min = pd.read_csv(os.path.join('input','WDOFUT_F_0_15min.train.csv'), sep=';') 

data_daily = csv_diario.to_dict('records')
data_15min = csv_15min.to_dict('records')

data_daily = data_daily[1:]
data_15min = data_15min[1:]

In [3]:
def convert(x):
    return float(x.replace(',','.'))

def convert_date(x):
    tokens = x.split('/')
    return tokens[2] + tokens[1] + tokens[0]

def convert_time(x):
    tokens = x.split(":")
    return int(tokens[0] + tokens[1])

In [4]:


data_15min = [{'Asset': d['Asset'],
               'Date': convert_date(d['Date']), 
               'Time': convert_time(d['Time']), 
               'Open': convert(d['Open']), 
               'High': convert(d['High']), 
               'Low': convert(d['Low']), 
               'Close': convert(d['Close']), 
               'Volume': convert(d['Volume']),
               'Trades': d['Trades']}
               for d in data_15min]
data_daily = [{'Asset': d['Asset'],
               'Date': convert_date(d['Date']), 
               'Open': convert(d['Open']), 
               'High': convert(d['High']), 
               'Low': convert(d['Low']), 
               'Close': convert(d['Close']), 
               'Volume': convert(d['Volume']),
               'Trades': d['Trades']}
               for d in data_daily]

sorted(data_15min, key=lambda x: x['Date'], reverse=True)
sorted(data_daily, key=lambda x: x['Date'], reverse=True)

days_in_15min = sorted(set([x['Date'] for x in data_15min]))
days_in_daily = sorted(set([x['Date'] for x in data_daily]))

data_daily = [x for x in data_daily if x['Date'] in days_in_15min]

In [5]:
returned_coefs = []

for i, current_day in enumerate(data_daily):
    if i > 3:
        first_open = data_daily[i-3]['Open']

        current_date = data_daily[i]['Date']

        open_x1 = 1
        high_x1 = ((data_daily[i-3]['High'] / first_open) - 1) * 100 
        close_x1 = ((data_daily[i-3]['Close'] /first_open) - 1) * 100
        low_x1 = ((data_daily[i-3]['Low'] / first_open) - 1) * 100

        last_close = data_daily[i-3]['Close']

        open_x2 = ((data_daily[i-2]['Open'] / last_close) -1) * 100
        high_x2 = ((data_daily[i-2]['High'] / last_close) -1) * 100
        close_x2 = ((data_daily[i-2]['Close'] / last_close) -1) * 100
        low_x2 = ((data_daily[i-2]['Low'] / last_close) -1) * 100

        last_close = data_daily[i-2]['Close']

        open_x3 = ((data_daily[i-1]['Open'] / last_close) -1) * 100
        high_x3 = ((data_daily[i-1]['High'] / last_close) -1) * 100
        close_x3 = ((data_daily[i-1]['Close'] / last_close) -1) * 100
        low_x3 = ((data_daily[i-1]['Low'] / last_close) -1) * 100

        last_close = data_daily[i-1]['Close']

        current_open = ((data_daily[i]['Open'] / last_close) - 1) * 100

        current_values = [open_x1, high_x1, close_x1, low_x1, open_x2, high_x2, close_x2,
                          low_x2, open_x3, high_x3, close_x3, low_x3, current_open]

        current_values_abs=[abs(x) for x in current_values]

        returned_coefs.append({
            'date': current_date,
            'first_open': first_open,

            'open_x1': open_x1,
            'high_x1': high_x1,
            'close_x1': close_x1,
            'low_x1': low_x1,

            'open_x2': open_x2,
            'high_x2': high_x2,
            'close_x2': close_x2,
            'low_x2': low_x2,

            'open_x3': open_x3,
            'high_x3': high_x3,
            'close_x3': close_x3,
            'low_x3': low_x3,

            'current_open' : current_open,
            'values': current_values,
            'absvalues': current_values_abs
            })

In [6]:
for current_x in returned_coefs:
    img = Image.new("RGB", (210,200), (255,255,255))
    img1 = ImageDraw.Draw(img)
    max_y = max(current_x['absvalues'])

    current_open = 100
    current_close = current_x['close_x1']
    negative = current_close > 0
    current_y = abs(current_close) / max_y
    current_y_pos = 100 - (current_y * 100) if negative else 100 + (current_y * 100)
    img1.line(((5 + (55*0),100), (50 + (55*0), current_y_pos )), (255,0,0), 2)
    img1.ellipse(((45 + (55*0),current_y_pos - 5),(55 + (55*0),current_y_pos + 5)), (255,0,0), 2)

    current_high = current_x['high_x1']
    current_low = current_x['low_x1']
    negative_high = current_high > 0
    negative_low = current_low > 0
    current_y_high = abs(current_high) / max_y
    current_y_low = abs(current_low) / max_y
    current_y_high_pos = 100 - (current_y_high * 100) if negative_high else 100 + (current_y_high * 100)
    current_y_low_pos = 100 - (current_y_low * 100) if negative_low else 100 + (current_y_low * 100)
    img1.line(((20 + (55*0),current_y_low_pos), (40 + (55*0), current_y_high_pos)), (255,0,0), 2)

    current_open = current_x['open_x2']
    current_close = current_x['close_x2']
    negative = current_close > 0
    current_y = abs(current_close) / max_y
    current_y_pos = 100 - (current_y * 100) if negative else 100 + (current_y * 100)
    current_y_open = abs(current_open) / max_y
    current_y_pos_open = 100 - (current_y_open * 100) if negative else 100 + (current_y_open * 100)
    img1.line(((5 + (55*1),current_y_pos_open), (50 + (55*1), current_y_pos )), (255,0,0), 2)
    img1.ellipse(((45 + (55*1),current_y_pos - 5),(55 + (55*1),current_y_pos + 5)), (255,0,0), 2)

    current_high = current_x['high_x2']
    current_low = current_x['low_x2']
    negative_high = current_high > 0
    negative_low = current_low > 0
    current_y_high = abs(current_high) / max_y
    current_y_low = abs(current_low) / max_y
    current_y_high_pos = 100 - (current_y_high * 100) if negative_high else 100 + (current_y_high * 100)
    current_y_low_pos = 100 - (current_y_low * 100) if negative_low else 100 + (current_y_low * 100)
    img1.line(((20 + (55*1),current_y_low_pos), (40 + (55*1), current_y_high_pos)), (255,0,0), 2)

    current_open = current_x['open_x3']
    current_close = current_x['close_x3']
    negative = current_close > 0
    current_y = abs(current_close) / max_y
    current_y_pos = 100 - (current_y * 100) if negative else 100 + (current_y * 100)
    current_y_open = abs(current_open) / max_y
    current_y_pos_open = 100 - (current_y_open * 100) if negative else 100 + (current_y_open * 100)
    img1.line(((5 + (55*2),current_y_pos_open), (50 + (55*2), current_y_pos )), (255,0,0), 2)
    img1.ellipse(((45 + (55*2),current_y_pos - 5),(55 + (55*2),current_y_pos + 5)), (255,0,0), 2)

    current_high = current_x['high_x3']
    current_low = current_x['low_x3']
    negative_high = current_high > 0
    negative_low = current_low > 0
    current_y_high = abs(current_high) / max_y
    current_y_low = abs(current_low) / max_y
    current_y_high_pos = 100 - (current_y_high * 100) if negative_high else 100 + (current_y_high * 100)
    current_y_low_pos = 100 - (current_y_low * 100) if negative_low else 100 + (current_y_low * 100)
    img1.line(((20 + (55*2),current_y_low_pos), (40 + (55*2), current_y_high_pos)), (255,0,0), 2)

    current_open = current_x['current_open']
    current_close = current_x['current_open']
    negative = current_close > 0
    current_y = abs(current_close) / max_y
    current_y_pos = 100 - (current_y * 100) if negative else 100 + (current_y * 100)
    current_y_open = abs(current_open) / max_y
    current_y_pos_open = 100 - (current_y_open * 100) if negative else 100 + (current_y_open * 100)
    img1.ellipse(((15 + (55*3),current_y_pos - 5),(25 + (55*3),current_y_pos + 5)), (0,255,0), 2)

    current_date = current_x['date']
    img.save(f'output\\{current_date}.png')

SyntaxError: f-string: unmatched '[' (Temp/ipykernel_12344/836949575.py, line 73)